# By @Christine Yang
# Extraction of tweets against the 'URL' of scientific papers.

In [0]:
#!/Users/christine/anaconda3/bin/python
# -*- coding: utf-8 -*-

import os, re, pandas as pd, time
import tweepy as tw
from tweepy.auth import OAuthHandler

consumer_key = ''
consumer_secret = ''
access_token_key = ''
access_token_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# SEARCH TWEETS
def search_article(query):
    query = '"'+query+'"'
    try:
        # get 999 tweets since Oct. 1 2019, untruncated
        tweets = tw.Cursor(api.search, q=query, since="2019-10-01", tweet_mode='extended').items(999)
        tweets = [tweet.full_text for tweet in tweets]
        print(len(tweets), query) # progress
        return tweets
    except:
        print('ERROR', query)
        return []

# GET LIST OF ARTICLES
df = pd.read_csv('articles_vaccine_covid2019.csv', encoding='utf-8-sig')
links = df['URL for article']
titles = df['Article name']
titles = [re.sub(r' *\[.*\] *', '', t) for t in titles] # remove stuff in brackets
articles = [(titles[i], links[i]) for i in range(len(links))] # [(title, link), ...]

# SEARCH FOR ARTICLE MENTIONS ON TWITTER
tweets, article_titles, article_links, found_by = [], [], [], []
i = 0
for title, link in articles[i:]:
    # search for article title
    t = search_article(title)
    tweets.extend(t)
    article_titles.extend([title]*len(t))
    article_links.extend([link]*len(t))
    found_by.extend(['article_title']*len(t))
    
    # search for article link
    t = search_article('url:'+link)
    tweets.extend(t)
    article_titles.extend([title]*len(t))
    article_links.extend([link]*len(t))
    found_by.extend(['article_link']*len(t))
    # checkpoint save
    if i % 10 == 0:
        df = pd.DataFrame({'tweet':tweets, 'article_title':article_titles, 'article_link':article_links, 'found_by':found_by})
        df = df.drop_duplicates()
        df.to_csv('result.csv', index=False, encoding='utf-8-sig')
    print(i) # progress
    i += 1
    time.sleep(10) # wait a bit

# SAVE RESULTS TO CSV
print(len(tweets))
df = pd.DataFrame({'tweet':tweets, 'article_title':article_titles, 'article_link':article_links, 'found_by':found_by})
df = df.drop_duplicates()
df.to_csv('VT_tweets.csv', index=False, encoding='utf-8-sig')
